In [59]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pylast
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import json
import random

In [60]:
playlist_features = []
valid_data_indices = []
for i in range(0, 100):
    feature_sing = pd.read_csv("./playlists/df_playlist_features_" + str(i) + ".csv")
    feature_sing = feature_sing.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    feature_sing = feature_sing.to_numpy()
    num_songs = feature_sing.shape[0]
    if (num_songs >= 30):
        valid_data_indices.append(i)
        playlist_features.append(feature_sing[:30, ].reshape((30, 13, 1)))
playlist_features = np.array(playlist_features)
print(len(valid_data_indices))
playlist_features.shape

67


(67, 30, 13, 1)

In [44]:
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# model.summary()


In [66]:
# read in labels
playlist_tags = []
with open('labels_100_playlists.json') as json_file:
    raw_label = json.load(json_file)
    for i in range(len(raw_label)):
        # filter labels with valid features
        if i in valid_data_indices:
            label = raw_label[i]
            label = np.array(list(label.values()))
#             label = label.reshape((-1, 1))
            playlist_tags.append(label)

    playlist_tags = np.array(playlist_tags)
    print(playlist_tags.shape)

(67, 274)


In [97]:
# train-test split get random indices
random.seed( 209 )
test_indices = random.sample(range(0, 66), 7)
print(test_indices)

# split features
train_features = []
test_features = []
for i in range(len(playlist_features)):
    if i in test_indices:
        test_features.append(playlist_features[i])
    else:
        train_features.append(playlist_features[i])

train_features = np.array(train_features)
test_features = np.array(test_features)

# split labels
train_labels = []
test_labels = []
for i in range(len(playlist_tags)):
    if i in test_indices:
        test_labels.append(playlist_tags[i])
    else:
        train_labels.append(playlist_tags[i])

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

[2, 21, 36, 5, 41, 43, 11]


In [98]:
# build CNN structure
input_shape = train_features[0].shape
num_category = train_labels[0].shape[0]

# model building
model = models.Sequential()
#convolutional layer with rectified linear unit activation
model.add(layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters with each size 3x3
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features by adding a pooling layer
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(layers.Dropout(0.25))
#flatten layer
model.add(layers.Flatten())
#fully connected to get all relevant data
model.add(layers.Dense(128, activation='relu'))
#one more dropout for convergence
model.add(layers.Dropout(0.5))
#output multiple sigmoid to generate output probabilities
model.add(layers.Dense(num_category, activation='sigmoid'))

model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 28, 11, 32)        320       
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 26, 9, 64)         18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 4, 64)         0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 13, 4, 64)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 3328)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               426112    
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)             

In [99]:
# Use adaDelta as the adaptive learning rate
model.compile(loss=tf.keras.losses.squared_hinge,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [100]:
# Train Model
# too few data to have a batch_size
#batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(train_features, train_labels,
          epochs=num_epoch,
          verbose=1,
          validation_data=(test_features, test_labels))

Train on 60 samples, validate on 7 samples
Epoch 1/10
60/60 [==============================] - 1s 8ms/sample - loss: nan - accuracy: 0.1000 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
60/60 [==============================] - 0s 908us/sample - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
60/60 [==============================] - 0s 828us/sample - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
60/60 [==============================] - 0s 849us/sample - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
60/60 [==============================] - 0s 821us/sample - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
60/60 [==============================] - 0s 826us/sample - loss: nan - accuracy: 0.1167 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
60/60 [==============================] - 0s 845us/sample - loss: nan - accuracy: 0.1167 - val_lo

In [101]:
score = model.evaluate(train_features, train_labels, verbose=1)
print(score)
print('Train loss:', score[0]) 
print('Train accuracy:', score[1]) 

60/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================